<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci-551-lab-assignments-hw/blob/main/homework-5/solution/PySpark_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install pyspark
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=1aacc411c7d28f0ca6e4c8024be19c576fe141c4ee3c2d6baf1af7e6b27bab91
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.8 MB/s eta 0:00:00
time: 546 µs (started: 2023-04-20 08:54:09 +00:00)


In [2]:
%%bash
java --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
time: 373 ms (started: 2023-04-20 08:54:09 +00:00)


# Spark Session

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

time: 12.8 s (started: 2023-04-20 08:54:09 +00:00)


In [4]:
import pyspark.sql.functions as fc

time: 676 µs (started: 2023-04-20 08:58:03 +00:00)


# Assignment Solution

Load Data to dataframes

In [6]:
country_df = spark.read.json("country.json")
city_df = spark.read.json("city.json")
country_lang_df = spark.read.json("countrylanguage.json")

aqi_df = spark.read.csv("aqi.csv", header=True, inferSchema=True)

time: 1.85 s (started: 2023-04-20 09:06:24 +00:00)


### Sample View

In [12]:
country_df.show(5)

+-------+----+-----+-------------+------+------+--------------------+--------------------+---------+--------------+--------------------+-----------+----------+--------------------+-----------+
|Capital|Code|Code2|    Continent|   GNP|GNPOld|      GovernmentForm|         HeadOfState|IndepYear|LifeExpectancy|           LocalName|       Name|Population|              Region|SurfaceArea|
+-------+----+-----+-------------+------+------+--------------------+--------------------+---------+--------------+--------------------+-----------+----------+--------------------+-----------+
|    129| ABW|   AW|North America| 828.0| 793.0|Nonmetropolitan T...|             Beatrix|        0|          78.4|               Aruba|      Aruba|    103000|           Caribbean|      193.0|
|      1| AFG|   AF|         Asia|5976.0|   0.0|     Islamic Emirate|       Mohammad Omar|     1919|          45.9|Afganistan/Afqane...|Afghanistan|  22720000|Southern and Cent...|   652090.0|
|     56| AGO|   AO|       Africa|6

In [8]:
city_df.show(5)

+-----------+-------------+---+--------------+----------+
|CountryCode|     District| ID|          Name|Population|
+-----------+-------------+---+--------------+----------+
|        AFG|        Kabol|  1|         Kabul|   1780000|
|        AFG|     Qandahar|  2|      Qandahar|    237500|
|        AFG|        Herat|  3|         Herat|    186800|
|        AFG|        Balkh|  4|Mazar-e-Sharif|    127800|
|        NLD|Noord-Holland|  5|     Amsterdam|    731200|
+-----------+-------------+---+--------------+----------+
only showing top 5 rows

time: 330 ms (started: 2023-04-20 09:07:22 +00:00)


In [9]:
country_lang_df.show(5)

+-----------+----------+----------+----------+
|CountryCode|IsOfficial|  Language|Percentage|
+-----------+----------+----------+----------+
|        ABW|         T|     Dutch|       5.3|
|        ABW|         F|   English|       9.5|
|        ABW|         F|Papiamento|      76.7|
|        ABW|         F|   Spanish|       7.4|
|        AFG|         F|   Balochi|       0.9|
+-----------+----------+----------+----------+
only showing top 5 rows

time: 317 ms (started: 2023-04-20 09:07:38 +00:00)


In [10]:
aqi_df.show(5)

+----------+---------+--------------------+-----+
|      date|  country|              status|value|
+----------+---------+--------------------+-----+
|2022-07-21|  Albania|                Good|   14|
|2022-07-21|  Algeria|            Moderate|   65|
|2022-07-21|  Andorra|            Moderate|   55|
|2022-07-21|   Angola|Unhealthy for Sen...|  113|
|2022-07-21|Argentina|            Moderate|   63|
+----------+---------+--------------------+-----+
only showing top 5 rows

time: 317 ms (started: 2023-04-20 09:07:53 +00:00)


## Question 2

### a.i

In [20]:
joined_df = aqi_df.join(country_df, aqi_df.country == country_df.Name)
joined_df.show(10)

+----------+----------+--------------------+-----+-------+----+-----+-------------+--------+--------+--------------------+--------------------+---------+--------------+--------------------+----------+----------+--------------------+-----------+
|      date|   country|              status|value|Capital|Code|Code2|    Continent|     GNP|  GNPOld|      GovernmentForm|         HeadOfState|IndepYear|LifeExpectancy|           LocalName|      Name|Population|              Region|SurfaceArea|
+----------+----------+--------------------+-----+-------+----+-----+-------------+--------+--------+--------------------+--------------------+---------+--------------+--------------------+----------+----------+--------------------+-----------+
|2022-07-21|   Albania|                Good|   14|     34| ALB|   AL|       Europe|  3205.0|  2500.0|            Republic|      Rexhep Mejdani|     1912|          71.6|          ShqipÃ«ria|   Albania|   3401200|     Southern Europe|    28748.0|
|2022-07-21|   Alger

In [17]:
result_df = joined_df.select("Name").distinct()
result_df.show()

+-----------+
|       Name|
+-----------+
|       Chad|
|      Macao|
|    Senegal|
|     Sweden|
|Philippines|
|   Malaysia|
|  Singapore|
|     Turkey|
|       Iraq|
|    Germany|
|   Cambodia|
|     Jordan|
|      Sudan|
|     France|
|     Greece|
|  Sri Lanka|
|     Taiwan|
|    Algeria|
|       Togo|
|   Slovakia|
+-----------+
only showing top 20 rows

time: 1.2 s (started: 2023-04-20 09:18:47 +00:00)


### a.ii

In [18]:
country_set = country_df.select("Name").distinct()
aqi_set = aqi_df.select("country").distinct()

result_set = country_set.intersect(aqi_set)

result_set.show()

+-----------+
|       Name|
+-----------+
|       Chad|
|      Macao|
|    Senegal|
|     Sweden|
|Philippines|
|   Malaysia|
|  Singapore|
|     Turkey|
|       Iraq|
|    Germany|
|   Cambodia|
|     Jordan|
|      Sudan|
|     France|
|     Greece|
|  Sri Lanka|
|     Taiwan|
|    Algeria|
|       Togo|
|   Slovakia|
+-----------+
only showing top 20 rows

time: 1.3 s (started: 2023-04-20 09:21:25 +00:00)


### b.i